# Min cost 
We start with label correcting shortest path to find negative cycles

In [29]:
immutable Arc
    from::Int
    to::Int
    cost::Int
end

In [30]:
adjL=[[Arc(1,2,2),Arc(1,5,5)],
    [Arc(2,3,7),Arc(2,4,2)],
    [Arc(3,4,-5)],
    Arc[],
    [Arc(5,2,-4),Arc(5,4,1)]
]

5-element Array{Array{Arc,1},1}:
 Arc[Arc(1, 2, 2), Arc(1, 5, 5)] 
 Arc[Arc(2, 3, 7), Arc(2, 4, 2)] 
 Arc[Arc(3, 4, -5)]              
 Arc[]                           
 Arc[Arc(5, 2, -4), Arc(5, 4, 1)]

In [31]:
# Some experiments with views 
d=[0, 5, 10000, 1, 10000]
S̄=[2,3,4,5]
S̄[indmin(d[S̄])]
#@time d[S̄]
#@time view(d,S̄)
d=rand(1:1000,10000)
inds=randperm(10000)[1:1000];

In [32]:
@time d[inds];

  0.000013 seconds (6 allocations: 8.109 KiB)


In [33]:
@time view(d,inds);

  0.000015 seconds (12 allocations: 320 bytes)


In [35]:
# A very bad Djesktra 
function simpleShortestPath(s,adjL)
    n=length(adjL) #O(1)
    pred=zeros(Int,n)  #O(n)
    S=Int[]    # O(1)
    S̄=[1:n...]  #O(n)
    d=ones(Int,n)*typemax(Int) # Some big number  O(n)
    d[s]=0 #(O(1))
    while length(S)<n   # The loop will repeat n times
        i=S̄[indmin(view(d,S̄))] #O(n)
        push!(S,i) #O(1)
        deleteat!(S̄,findfirst(S̄.==i)) #If S is an array or list, this is O(n)
        for arc in adjL[i]
            j=arc.to
            new_d=d[i]+arc.cost
            if d[j]>new_d
                d[j]=new_d
                pred[j]=i
            end
        end
    end
    pred, d
end

simpleShortestPath (generic function with 1 method)

In [36]:
@time simpleShortestPath(1,adjL)

  0.548350 seconds (241.09 k allocations: 12.434 MiB, 1.02% gc time)


([0, 5, 2, 2, 1], [0, 1, 9, 4, 5])

In [6]:
function lcsp(s,adjL) # label correcting shortest path 
    n=length(adjL) #O(1)
    pred=zeros(Int,n)  #O(n)
    LIST=[s]
    d=ones(Int,n)*typemax(Int) # Some big number  O(n)
    d[s]=0 #(O(1))
    while !isempty(LIST)
        i=pop!(LIST) # Shift would have also worked
        for arc in adjL[i]
            j=arc.to
            new_d=d[i]+arc.cost
            if d[j]>new_d
                d[j]=new_d
                pred[j]=i
                push!(LIST,j)
            end
        end
    end
    pred, d
end

lcsp (generic function with 1 method)

In [7]:
lcsp(1,adjL)

([0,5,2,2,1],[0,1,8,3,5])

Dijsktra did not give us the right answer here, hte lcsp did!

Now we write the FIFO version of LCSP

In [41]:
L=[1]
push!(L,2)
shift!(L)

1

In [42]:
L=[1]
unshift!(L,2)
pop!(L)

1

In [43]:
function lcspFIFO(s,adjL) # label correcting shortest path 
    n=length(adjL) #O(1)
    pred=zeros(Int,n)  #O(n)
    LIST=[s]
    d=ones(Int,n)*typemax(Int) # Some big number  O(n)
    d[s]=0 #(O(1))
    while !isempty(LIST)
        i=shift!(LIST) # shift here to be FIFO
        for arc in adjL[i]
            j=arc.to
            new_d=d[i]+arc.cost
            if d[j]>new_d
                d[j]=new_d
                pred[j]=i
                push!(LIST,j)
            end
        end
    end
    pred, d
end

lcspFIFO (generic function with 1 method)

In [44]:
lcspFIFO(1,adjL)

([0, 5, 2, 2, 1], [0, 1, 8, 3, 5])

We make a slight modification to have a -ve cycle

In [45]:
adjL=[[Arc(1,2,2),Arc(1,5,5)],
    [Arc(2,3,7)],
    [Arc(3,4,-5)],
    [Arc(4,2,-3)],
    [Arc(5,2,-4),Arc(5,4,1)]
]

5-element Array{Array{Arc,1},1}:
 Arc[Arc(1, 2, 2), Arc(1, 5, 5)] 
 Arc[Arc(2, 3, 7)]               
 Arc[Arc(3, 4, -5)]              
 Arc[Arc(4, 2, -3)]              
 Arc[Arc(5, 2, -4), Arc(5, 4, 1)]

In [16]:
function negativeCycleDetect(s,adjL) # label correcting shortest path 
    n=length(adjL) #O(1)
    pred=zeros(Int,n)  #O(n)
    LIST=[s]
    d=ones(Int,n)*typemax(Int) # Some big number  O(n)
    d[s]=0 #(O(1))
    p_counter=zeros(Int,n)
    cycle=Int[]
    key_node=0
    while !isempty(LIST)
        i=shift!(LIST) # shift here to be FIFO
        for arc in adjL[i]
            j=arc.to
            new_d=d[i]+arc.cost
            if d[j]>new_d
                d[j]=new_d
                pred[j]=i
                push!(LIST,j)
                p_counter[j]+=1
                if p_counter[j]>=n-1 
                    key_node=j
                    break
                end
            end
        end
        (key_node>0) && break
    end
    pred, d, key_node
end

negativeCycleDetect (generic function with 1 method)

In [18]:
pred,d,key_node=negativeCycleDetect(1,adjL)

([0,4,2,3,1],[0,0,7,2,5],4)

In [20]:
cycle=[key_node]
j=pred[key_node]
while j!=key_node
    unshift!(cycle,j)
    j=pred[j]
end
unshift!(cycle,key_node) #You dod't really have to do it
cycle

4-element Array{Int64,1}:
 4
 2
 3
 4

In [22]:
# We now want to get the cycle as a list of arcs
cycle_as_arcs=Arc[]
for i=1:length(cycle)-1
    from=cycle[i]
    to=cycle[i+1]
    for arc in adjL[from]
        if arc.to==to 
            push!(cycle_as_arcs,arc) #For min-cost I might just need the capacities not the full arcs
        end
    end
end
cycle_as_arcs

3-element Array{Arc,1}:
 Arc(4,2,-3)
 Arc(2,3,7) 
 Arc(3,4,-5)